In [40]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
import pandas as pd

In [41]:
spark = SparkSession.builder.appName("Internet_data").getOrCreate()


In [42]:
df = spark.read.option("header", True) \
                .option("inferSchema", True) \
                .csv("data/Internet_Fijo_Penetraci_n_Municipio.csv")

In [43]:
df.printSchema()

root
 |-- AÑO: integer (nullable = true)
 |-- TRIMESTRE: integer (nullable = true)
 |-- COD_DEPARTAMENTO: integer (nullable = true)
 |-- DEPARTAMENTO: string (nullable = true)
 |-- COD_MUNICIPIO: integer (nullable = true)
 |-- MUNICIPIO: string (nullable = true)
 |-- No. ACCESOS FIJOS A INTERNET: integer (nullable = true)
 |-- POBLACIÓN DANE: integer (nullable = true)
 |-- INDICE: string (nullable = true)



In [44]:
df.columns

['AÑO',
 'TRIMESTRE',
 'COD_DEPARTAMENTO',
 'DEPARTAMENTO',
 'COD_MUNICIPIO',
 'MUNICIPIO',
 'No. ACCESOS FIJOS A INTERNET',
 'POBLACIÓN DANE',
 'INDICE']

In [45]:
# Renombrar columnas para evitar problemas con espacios y caracteres especiales
df = df.withColumnRenamed('No. ACCESOS FIJOS A INTERNET', 'No_ACCESOS_FIJOS_A_INTERNET') \
       .withColumnRenamed('POBLACIÓN DANE', 'POBLACION_DANE')

In [46]:
# Variables a contar (usando los nuevos nombres)
variables = ['AÑO', 'TRIMESTRE', 'COD_DEPARTAMENTO', 'DEPARTAMENTO', 'COD_MUNICIPIO', 'MUNICIPIO', 
             'No_ACCESOS_FIJOS_A_INTERNET', 'POBLACION_DANE', 'INDICE']

In [47]:
# Contar registros por cada variable
for variable in variables:
    count = df.select(variable).distinct().count()
    print(f"Cantidad de registros únicos en {variable}: {count}")

Cantidad de registros únicos en AÑO: 4
Cantidad de registros únicos en TRIMESTRE: 4
Cantidad de registros únicos en COD_DEPARTAMENTO: 33
Cantidad de registros únicos en DEPARTAMENTO: 33
Cantidad de registros únicos en COD_MUNICIPIO: 1119
Cantidad de registros únicos en MUNICIPIO: 1034
Cantidad de registros únicos en No_ACCESOS_FIJOS_A_INTERNET: 4014
Cantidad de registros únicos en POBLACION_DANE: 4216
Cantidad de registros únicos en INDICE: 2196


In [48]:
filas = df.count()
filas

13370

In [49]:
years = df.select('AÑO').distinct().orderBy('AÑO').collect()
years

[Row(AÑO=2019), Row(AÑO=2020), Row(AÑO=2021), Row(AÑO=2022)]

In [50]:
df_regiones = pd.read_excel("data/subregiones.xls")

In [51]:
df_regiones

,NOMBRE_DEPTO,PROVINCIA,CODIGO_MUNICIPIO,NOMBRE_MPIO,Nombre,Total
0,AMAZONAS,AMAZONAS,91263.0,EL ENCANTO,El Encanto,NaN
1,NaN,NaN,91405.0,LA CHORRERA,La Chorrera,NaN
2,NaN,NaN,91407.0,LA PEDRERA,La Pedrera,NaN
3,NaN,NaN,91430.0,LA VICTORIA,La Victoria,NaN
4,NaN,NaN,91001.0,LETICIA,Leticia,NaN
...,...,...,...,...,...,...
1148,NaN,NaN,99524.0,LA PRIMAVERA,La Primavera,NaN
1149,NaN,NaN,99001.0,PUERTO CARREÑO,Puerto Carreño,NaN
1150,NaN,NaN,99624.0,SANTA ROSALÍA,Santa Rosalía,NaN
1151,Total VICHADA,NaN,NaN,NaN,NaN,4.0


In [52]:
# Eliminar filas que contengan 'Total' en la columna 'NOMBRE_DEPTO'
df_regiones = df_regiones[~df_regiones['NOMBRE_DEPTO'].str.contains("Total", na=False)]
# Rellenar los valores faltantes en 'NOMBRE_DEPTO' y 'PROVINCIA' con los valores anteriores
df_regiones['NOMBRE_DEPTO'] = df_regiones['NOMBRE_DEPTO'].fillna(method='ffill')
df_regiones['PROVINCIA'] = df_regiones['PROVINCIA'].fillna(method='ffill')
# Eliminar la columna 'Total' si no es necesaria
df_regiones = df_regiones.drop(columns=['Total'])


/tmp/ipykernel_25875/3452811582.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_regiones['NOMBRE_DEPTO'] = df_regiones['NOMBRE_DEPTO'].fillna(method='ffill')
/tmp/ipykernel_25875/3452811582.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regiones['NOMBRE_DEPTO'] = df_regiones['NOMBRE_DEPTO'].fillna(method='ffill')
/tmp/ipykernel_25875/3452811582.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_regiones['PROVINCIA'] = df_regiones['PROVINCIA'].fillna(method='ffill')
/tmp/ipykernel_25875/3452811582.py:5: SettingWithCopyWarning: 
A value is trying to be se

In [53]:
df_regiones

,NOMBRE_DEPTO,PROVINCIA,CODIGO_MUNICIPIO,NOMBRE_MPIO,Nombre
0,AMAZONAS,AMAZONAS,91263.0,EL ENCANTO,El Encanto
1,AMAZONAS,AMAZONAS,91405.0,LA CHORRERA,La Chorrera
2,AMAZONAS,AMAZONAS,91407.0,LA PEDRERA,La Pedrera
3,AMAZONAS,AMAZONAS,91430.0,LA VICTORIA,La Victoria
4,AMAZONAS,AMAZONAS,91001.0,LETICIA,Leticia
...,...,...,...,...,...
1145,VAUPES,VAUPES,97889.0,YAVARATÉ,Yavaraté
1147,VICHADA,VICHADA,99773.0,CUMARIBO,Cumaribo
1148,VICHADA,VICHADA,99524.0,LA PRIMAVERA,La Primavera
1149,VICHADA,VICHADA,99001.0,PUERTO CARREÑO,Puerto Carreño


In [54]:
df_regiones.to_csv("data/subregiones.csv", index=False)